In [ ]:
import tweepy
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from wordcloud import WordCloud

#Initiate the security token
consumer_key = "JoPmGvLluEyLqyASsz49_____"
consumer_secret = "sOcYXKBtnWYtIkzjiYS2tw9y7OdeX2qcP3SnZk8FYe5kz_____"
access_token = "1287763964466610176-K64Z6DDuzE8zLWvH0vn5Q8Nz6_____"
access_token_secret = "ZLA10vZWo27ervw0DgCGLKpWsZ35GtEcmC9ploOq_____"

# Accesing Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Take the trending topic in Indonesia 
Indonesia_woe = 23424846
trends = api.trends_place(Indonesia_woe)
print('\nDaftar Trending Saat Ini di Indonesia :')
for item in (trends[0]['trends']):
       print (item['name'])

# Get the trending topics into a list
populer=[]
for item in (trends[0]['trends']):
       populer.append(item['name'])
print ([list((i, populer[i])) for i in range(len(populer))]) 

# Choose a trending topic
search_words = "Setneg"
date_since = "2020-10-21"
new_search = search_words + " -filter:retweets"
tweets = tweepy.Cursor(api.search, q=new_search,
                           lang="id",
                           since=date_since).items()

# Get the text for each username and get it into Dataframe 
users_locs = [[tweet.user.screen_name, tweet.text] for tweet in tweets]
df_text = pd.DataFrame(data=users_locs, 
                    columns=['user', 'tweet'])

# Save the DataFrame into database
engine = create_engine('sqlite:///Twitter_analysis.db', echo=True)
sqlite_connection = engine.connect()
df_text.to_sql('data_tweet', con=engine)

# Cleaning the tweet text
tweets = tweepy.Cursor(api.search, q=new_search,
                           lang="id",
                           since=date_since).items()
items = []
for tweet in tweets:
    items.append (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet.text).split()))

# Open files that contain positive and negative words
pos_list= open("./kata_positif.txt","r")
pos_kata = pos_list.readlines()
neg_list= open("./kata_negatif.txt","r")
neg_kata = neg_list.readlines()

# Doing sentiment analysis
hasil = []
for item in items:
    count_p = 0
    count_n = 0
    for kata_pos in pos_kata:
        if kata_pos.strip() in item:
            count_p +=1
    for kata_neg in neg_kata:
        if kata_neg.strip() in item:
            count_n +=1
    print ("positif: "+str(count_p))
    print ("negatif: "+str(count_n))
    hasil.append(count_p - count_n)
print ("Nilai rata-rata: "+str(np.mean(hasil)))
print ("Standar deviasi: "+str(np.std(hasil)))

# Plot the sentiment analysis result
labels, counts = np.unique(hasil, return_counts=True)
plt.bar(labels, counts, align='center')
plt.gca().set_xticks(labels)
plt.show()

# Plot the top 5 twitter username that make tweets 
df_text['user'].value_counts().head().plot(kind='bar')

# Plot the Wordcloud
text = df_text['tweet'].values
wordcloud = WordCloud(width=1600, height=800, max_font_size=200).generate(str(text))
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()